In [2]:
!pip install transformers
!pip install torch
!pip install sckikit-learn
!pip install --upgrade transformers


ERROR: Could not find a version that satisfies the requirement sckikit-learn (from versions: none)
ERROR: No matching distribution found for sckikit-learn


     ---------------------------------------- 7.4/7.4 MB 29.5 MB/s eta 0:00:00
     ---------------------------------------- 266.3/266.3 kB ? eta 0:00:00
     ---------------------------------------- 268.8/268.8 kB ? eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.10.1
    Uninstalling huggingface-hub-0.10.1:
      Successfully uninstalled huggingface-hub-0.10.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.24.0
    Uninstalling transformers-4.24.0:
      Successfully uninstalled transformers-4.24.0


In [3]:
import transformers
import torch
import sklearn
import pandas as pd

# Step 1: Import Libraries and Load IMDb Data

In [4]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.model_selection import train_test_split
import torch
import torch.nn.functional as F
import torch.optim as optim

# Path to your CSV file
file_path = r"C:\Users\gonza\OneDrive\Desktop\IMDB_Dataset.csv"

# Read the CSV file into a DataFrame
data = pd.read_csv(file_path)

# Assuming the columns are named 'review' and 'sentiment'
texts = data['review']
labels = data['sentiment'].apply(lambda x: 1 if x == 'positive' else 0) # Convert to binary labels

# Split your data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2)

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')



C:\Users\gonza\anaconda3\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\gonza\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


# Step 2: Tokenize and Prepare Data

In [5]:
# Tokenize the training and validation texts
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True)
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True)

# Convert to PyTorch tensors
train_input_ids = torch.tensor(train_encodings['input_ids'])
train_attention_mask = torch.tensor(train_encodings['attention_mask'])
train_labels_tensor = torch.tensor(train_labels.tolist())

val_input_ids = torch.tensor(val_encodings['input_ids'])
val_attention_mask = torch.tensor(val_encodings['attention_mask'])
val_labels_tensor = torch.tensor(val_labels.tolist())

# Create the TensorDatasets
train_dataset = TensorDataset(train_input_ids, train_attention_mask, train_labels_tensor)
val_dataset = TensorDataset(val_input_ids, val_attention_mask, val_labels_tensor)

# Create the DataLoaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

# Step 3: Load Pre-trained BERT Model

In [6]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Step 4: Train the Model

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

optimizer = optim.AdamW(model.parameters(), lr=1e-5)

for epoch in range(3):  # Number of epochs
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    # You can add validation logic here


In [ ]:
 # Validation logic
model.eval()
val_loss = 0
correct_predictions = 0
with torch.no_grad():
    for batch in val_loader:
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        val_loss += outputs.loss.item()
        preds = torch.argmax(outputs.logits, dim=1)
        correct_predictions += (preds == labels).sum().item()

val_accuracy = correct_predictions / len(val_texts)
print(f"Validation Loss: {val_loss / len(val_loader)}")
print(f"Validation Accuracy: {val_accuracy}")

# Step 5: Evaluate the Model

In [9]:

import pandas as pd


In [10]:


# Path to your CSV file
file_path = r"C:\Users\EXG0TSB\OneDrive - NEE\Documents\IMDB Dataset.csv"

# Read the CSV file into a DataFrame
data = pd.read_csv(file_path)

# Print the first few rows to verify the content
print(data.head())



                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [9]:
data.head()


,Respondent ID,Collector ID,Start Date,End Date,IP Address,Email Address,"On a scale of 1 to 10, how well does the PUR software help you do your job?",What is the main reason for your rating?,What is the best/worst change/feature added in the past 6 months?
0,NaN,NaN,NaT,NaT,NaN,NaN,Response,Open-Ended Response,Open-Ended Response
1,1.183862e+11,451942577.0,2023-08-03 09:48:07,2023-08-03 09:57:57,165.225.223.3,NaN,7,I use PUR in a supporting role. I use it for d...,Worst: The target start and finish dates
2,1.183862e+11,451942577.0,2023-08-03 09:18:50,2023-08-03 09:24:11,104.129.207.34,NaN,Extremely likely - 10,The new changes which were added to PUR makes ...,"Best Changes: Schedule tab changes, EOJC auto ..."
3,1.183862e+11,451942577.0,2023-08-03 08:56:17,2023-08-03 09:00:19,165.225.222.230,NaN,Extremely likely - 10,"Program does what it was designed to do, users...",I wish we would still have a field for the old...
4,1.183861e+11,451942577.0,2023-08-03 06:29:52,2023-08-03 06:33:48,165.225.37.9,NaN,5,There needs to be a formal training and an off...,"I wasn't aware of any updates recently. Again,..."


In [10]:
# Removing the unnecessary first row (index 0)
cleaned_data = data.drop(index=0)

# Displaying the first few rows to confirm that the unnecessary row has been removed
cleaned_data.head()


,Respondent ID,Collector ID,Start Date,End Date,IP Address,Email Address,"On a scale of 1 to 10, how well does the PUR software help you do your job?",What is the main reason for your rating?,What is the best/worst change/feature added in the past 6 months?
1,1.183862e+11,451942577.0,2023-08-03 09:48:07,2023-08-03 09:57:57,165.225.223.3,NaN,7,I use PUR in a supporting role. I use it for d...,Worst: The target start and finish dates
2,1.183862e+11,451942577.0,2023-08-03 09:18:50,2023-08-03 09:24:11,104.129.207.34,NaN,Extremely likely - 10,The new changes which were added to PUR makes ...,"Best Changes: Schedule tab changes, EOJC auto ..."
3,1.183862e+11,451942577.0,2023-08-03 08:56:17,2023-08-03 09:00:19,165.225.222.230,NaN,Extremely likely - 10,"Program does what it was designed to do, users...",I wish we would still have a field for the old...
4,1.183861e+11,451942577.0,2023-08-03 06:29:52,2023-08-03 06:33:48,165.225.37.9,NaN,5,There needs to be a formal training and an off...,"I wasn't aware of any updates recently. Again,..."
5,1.183857e+11,451942577.0,2023-08-02 17:28:48,2023-08-02 17:31:50,165.225.222.245,NaN,7,The system is hard to navigate.,No answer


In [11]:
# Removing the "Email Address" column as it does not contain any useful entries
cleaned_data = cleaned_data.drop(columns=["Email Address"])

# Checking for missing values in the cleaned dataset
missing_values_summary = cleaned_data.isnull().sum()
missing_values_summary


Respondent ID                                                                   0
Collector ID                                                                    0
Start Date                                                                      0
End Date                                                                        0
IP Address                                                                      0
On a scale of 1 to 10, how well does the PUR software help you do your job?     0
What is the main reason for your rating?                                        9
What is the best/worst change/feature added in the past 6 months?              36
dtype: int64